In [ ]:
%pip install streamlit autogen-agentchat~=0.2 groq autogen pathlib typing matplotlib tavily-python yfinance seaborn matplotlib mplfinance

In [ ]:
import os
import json
import requests
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Annotated, List
from autogen import AssistantAgent, UserProxyAgent, register_function
from autogen.coding import LocalCommandLineCodeExecutor
from tavily import TavilyClient
import seaborn as sns
import matplotlib.pyplot as plt
import mplfinance as mpf
from tavily import TavilyClient
import yfinance as yf
from autogen import ConversableAgent
import streamlit as st

os.environ["GROQ_API_KEY"] = ""
tavily_client = TavilyClient(api_key="")

In [ ]:
# Configure Groq API
config_list = [{
    "model": "deepseek-r1-distill-llama-70b",
    "api_key": os.environ.get("GROQ_API_KEY"),
    "api_type": "groq"
}]

# Create a directory to store code files from the code executor
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

# Initialize the assistant FIRST
assistant = AssistantAgent(
    name="groq_assistant",
    system_message= """
                  You are a highly capable Agentic Generative AI with access to the following tools: `weather_forecast` (for fetching weather data), `generate_bar_chart` (for creating bar charts), `tavily_search` (for web searches), `fetch_stock_data` (for fetching historical stock data), `visualize_stock_data` (for generating advanced stock data visualizations). Your goal is to accurately answer the user's query, leveraging your tools effectively and reasoning step-by-step. Let's think step by step: 1. **Initial Understanding:** Fully understand the user's request. What information are they looking for? What is the desired output format? 2. **Tool Selection:** Determine which tools are most appropriate (`weather_forecast`, `generate_bar_chart`, `tavily_search`, `fetch_stock_data`, `visualize_stock_data`). Avoid unnecessary tool use. Consider the tool descriptions to choose the best tool. 3. **Step-by-Step Plan:** Outline a plan of action, breaking down the task into smaller steps, indicating which tool(s) to use for each step. 4. **Execution & Tool Calls:** Execute the plan, calling the necessary tools and recording the results. For example: If using `weather_forecast`, call it like this: `weather_forecast(location="London", unit="celsius")`. If using `tavily_search`, call it like this: `tavily_search(query="latest AI advancements")`. If using `generate_bar_chart`, call it like this: `generate_bar_chart(categories=["A", "B", "C"], values=[10, 20, 30], title="Example Chart")`. If using `fetch_stock_data`, call it like this: `fetch_stock_data(ticker="AAPL", start_date="2024-01-01")`. If using `visualize_stock_data`, call it like this: `visualize_stock_data(ticker="AAPL", start_date="2024-01-01")`. 5. **Synthesis & Response:** Synthesize the results from each step and provide a clear and concise answer to the user's query. Briefly summarize the steps taken and the tools used. Now, let's begin! **TASK:** Analyze the user's query (provided separately) and start executing the plan. Remember to think step-by-step, use the appropriate tools as needed, and follow the tool calling examples. Provide a comprehensive and well-reasoned answer. Say **TERMINATE** once you think the task was completed.
                   """,
    llm_config={"config_list": config_list},
)

def tavily_search(query: str) -> str:
    """Fetches search results using Tavily API."""
    try:
        response = tavily_client.search(query)
        return json.dumps(response, indent=2)
    except Exception as e:
        return f"Error fetching search results: {str(e)}"

def fetch_stock_data(ticker: str, start_date: str = "2024-01-01") -> str:
    """
    Fetches historical stock data for a given ticker symbol starting from the specified date.
    """
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date)
        if data.empty:
            return f"No data found for ticker symbol '{ticker}' starting from {start_date}."
        # Save the data to a CSV file
        file_path = work_dir / f"{ticker}_stock_data.csv"
        data.to_csv(file_path)
        return f"Stock data for {ticker} saved to {file_path}."
    except Exception as e:
        return f"An error occurred while fetching stock data: {str(e)}"

# Function to fetch real-time weather data
def get_current_weather(location: str, unit: str) -> str:
    """Fetches weather data from OpenWeatherMap API."""
    API_KEY = os.environ.get("OPENWEATHER_API_KEY")  # Ensure API key is set
    BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

    params = {
        "q": location,
        "appid": API_KEY,
        "units": "metric" if unit == "celsius" else "imperial"
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        return json.dumps({
            "location": data["name"],
            "temperature": data["main"]["temp"],
            "unit": unit
        })
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

# Define the weather forecasting function
def weather_forecast(
    location: Annotated[str, "City name"],
    unit: Annotated[str, "Temperature unit (fahrenheit/celsius)"] = "fahrenheit"
) -> str:
    """Fetches weather data from an external source."""
    weather_details = get_current_weather(location=location, unit=unit)
    weather = json.loads(weather_details)

    if weather["temperature"] == "unknown":
        return f"Could not retrieve weather for {weather['location']}."

    return f"{weather['location']} will be {weather['temperature']} degrees {weather['unit']}."

# Function to generate a bar chart
def generate_bar_chart(categories: List[str], values: List[int], title: str = "Bar Chart") -> str:
    """Generates a bar chart, saves it as an image, and displays it."""
    plt.figure(figsize=(8, 6))
    plt.bar(categories, values, color='skyblue')
    plt.xlabel("Categories")
    plt.ylabel("Values")
    plt.title(title)
    plt.xticks(rotation=45)

    # Save the plot and display it
    file_path = work_dir / "bar_chart.png"
    plt.savefig(file_path)
    plt.show()

    return f"Bar chart saved as {file_path} and displayed."

def visualize_stock_data(ticker: str, start_date: str = "2024-01-01") -> str:
    """
    Generates advanced visualizations for a given stock ticker, including a line plot of closing prices
    and a candlestick chart, starting from the specified date.
    """
    try:
        # Fetch stock data
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date)
        if data.empty:
            return f"No data found for ticker symbol '{ticker}' starting from {start_date}."

        # Set the plot style
        sns.set(style="whitegrid")

        # Create a line plot of the closing prices
        plt.figure(figsize=(14, 7))
        sns.lineplot(x=data.index, y=data['Close'], label='Closing Price', color='b')
        plt.title(f"{ticker} Closing Prices Since {start_date}")
        plt.xlabel("Date")
        plt.ylabel("Closing Price (USD)")
        plt.legend()
        plt.xticks(rotation=45)
        line_plot_path = work_dir / f"{ticker}_closing_prices.png"
        plt.savefig(line_plot_path)
        plt.close()

        # Create a candlestick chart
        mc = mpf.make_marketcolors(up='g', down='r', inherit=True)
        s = mpf.make_mpf_style(marketcolors=mc)
        candlestick_plot_path = work_dir / f"{ticker}_candlestick_chart.png"
        mpf.plot(data, type='candle', style=s, title=f"{ticker} Candlestick Chart", ylabel='Price (USD)',
                 savefig=dict(fname=candlestick_plot_path, dpi=100, bbox_inches='tight'))

        return f"Visualizations saved as {line_plot_path} and {candlestick_plot_path}."
    except Exception as e:
        return f"An error occurred while generating visualizations: {str(e)}"


# Create a user proxy agent with code execution capabilities
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: msg.get("content") and ("TERMINATE" in msg["content"] or "task completed" in msg["content"].lower()),
    human_input_mode="NEVER"
)

# Register the weather forecasting function explicitly
register_function(
    weather_forecast,
    caller=assistant,
    executor=user_proxy,
    name="weather_forecast",
    description="Fetch the weather forecast for a city in Fahrenheit or Celsius."
)

# Register the bar chart function
register_function(
    generate_bar_chart,
    caller=assistant,
    executor=user_proxy,
    name="generate_bar_chart",
    description="Generate a bar chart from given categories and values."
)

register_function(
    tavily_search,
    caller=assistant,
    executor=user_proxy,
    name="tavily_search",
    description="Search the web using Tavily API."
)

register_function(
    visualize_stock_data,
    caller=assistant,
    executor=user_proxy,
    name="visualize_stock_data",
    description="Generate advanced visualizations for a given stock ticker, including line and candlestick charts."
)


# register_function(
#     fetch_stock_data,
#     caller=assistant,
#     executor=user_proxy,
#     name="fetch_stock_data",
#     description="Fetch historical stock data for a given ticker symbol starting from a specified date."
# )


# Start the conversation with a structured prompt
user_proxy.initiate_chat(
    assistant,
    message=" Do you have access to visualize_stock_data visualization if you have demonstrate it "
)
